In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Seedstars_dataset_EDA_v0.csv', sep=';')

In [3]:
df.head()

,Unnamed: 0,company_id,country,industry,business_stage,incorporated,incorporation_date,Number of team members,accelerator,amount_to_raise,...,revenue_model_subscription_Imputed,revenue_model_freemium_Imputed,revenue_model_advertising_Imputed,revenue_model_licensing_Imputed,customer_type - B-to-B-to-C_Imputed,customer_type - B-to-B_Imputed,customer_type - B-to-C_Imputed,customer_type - B-to-G_Imputed,percent_increase_in_revenue,percent_increase_in_users
0,0,1,Uzbekistan,Consumer Services,Revenue Stage,0.0,2017,4.0,1.0,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,9.459459,9.782609
1,1,224,Uzbekistan,Travel and Transport,Development Stage,0.0,2017,4.0,0.0,30000,...,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,43.744532,69.230769
2,2,365,Uzbekistan,Financial Services,Beta Testing Stage,1.0,2019,1.0,1.0,3000,...,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,25000.000000,24.997500
3,3,440,Uzbekistan,Enterprise Services,Revenue Stage,0.0,2017,10.0,0.0,0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-14.446054,-20.588235
4,4,866,Uzbekistan,HR,Revenue Stage,1.0,2019,5.0,0.0,0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,-49.765258,-25.000000


In [4]:
feats = ['status_of_funding_STATUS',
 'country',
 'industry',
 'business_stage',
 'year_segment',
 'role_0',
 'country_0',
 'role_1']
df_final = pd.get_dummies(df,columns=feats,drop_first=True)

In [5]:
df_final.shape


(4244, 1826)

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X = df_final.drop(['website_active'],axis=1).values
y = df_final['website_active'].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Transforming the data

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
from sklearn.manifold import TSNE

In [11]:

#tsne = TSNE()
#tsne_data = tsne.fit_transform(scaler.fit_transform(num_data))
#X_train = tsne.fit_transform(X_train)
#X_test = tsne.fit_transform(X_test)

In [12]:
X_train.shape

(3395, 1825)

In [13]:
X_test.shape

(849, 1825)

# Building Artificial Neural Network

In [14]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
C:\Users\meral\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\meral\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\meral\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\meral\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [15]:
classifier = Sequential()

In [16]:
classifier.add(Dense(9, kernel_initializer = "uniform",activation = "relu", input_dim=1825))

In [17]:
classifier.add(Dense(1, kernel_initializer = "uniform",activation = "sigmoid"))

In [18]:
classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [19]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 1)


Epoch 1/1
3395/3395 [==============================] - 0s 97us/step - loss: 0.6205 - accuracy: 0.6851


# Running predictions on the Test Set

In [20]:
y_pred = classifier.predict(X_test)

In [21]:
#Since you’ve already trained the classifier with the training set, 
#this code will use the learning from the training process to make predictions on the test set. 
#This will give you the probabilities of website is active. You’ll work with a probability of 50% and above to indicate a high chance of the website activity the company.
y_pred = (y_pred > 0.5)

# Checking the Confusion Matrix

In [22]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[  0, 252],
       [  0, 597]], dtype=int64)

# Improving the Model Accuracy

In [23]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [24]:
def make_classifier():
    classifier = Sequential()
    classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=1825))
    classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
    classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])
    return classifier

In [25]:
classifier = KerasClassifier(build_fn = make_classifier, batch_size=10, nb_epoch=1)

In [26]:
accuracies = cross_val_score(estimator = classifier,X = X_train,y = y_train,cv = 10,n_jobs = -1)
 


In [27]:
mean = accuracies.mean()
mean

0.70104199051857

In [28]:
variance = accuracies.var()
variance

0.00018278310437086276

In [29]:
#You’ve improved your model’s accuracy by using K-Fold cross-validation. 
#In the next step, you’ll work on the overfitting problem.

# Adding Dropout Regularization to Fight Over-Fitting

In [30]:
from keras.layers import Dropout

classifier = Sequential()
classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=1825))
classifier.add(Dropout(rate = 0.1))
classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])

# Hyperparameter Tuning

In [31]:
from sklearn.model_selection import GridSearchCV
def make_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=1825))
    classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
    classifier.compile(optimizer= optimizer,loss = "binary_crossentropy",metrics = ["accuracy"])
    return classifier

In [32]:
classifier = KerasClassifier(build_fn = make_classifier)

In [33]:
params = {
    'batch_size':[20,35],
    'epochs':[2,3],
    'optimizer':['adam','rmsprop']
}

In [34]:
grid_search = GridSearchCV(estimator=classifier,
                           param_grid=params,
                           scoring="accuracy",
                           cv=2)

In [35]:
grid_search = grid_search.fit(X_train,y_train)

Epoch 1/2
1697/1697 [==============================] - 0s 98us/step - loss: 0.6484 - accuracy: 0.7012
Epoch 2/2
1697/1697 [==============================] - 0s 85us/step - loss: 0.5675 - accuracy: 0.7083
Epoch 1/2
1698/1698 [==============================] - 0s 112us/step - loss: 0.6544 - accuracy: 0.6631
Epoch 2/2
1698/1698 [==============================] - 0s 47us/step - loss: 0.5692 - accuracy: 0.6938
Epoch 1/2
1697/1697 [==============================] - 0s 99us/step - loss: 0.6646 - accuracy: 0.6612
Epoch 2/2
1697/1697 [==============================] - 0s 42us/step - loss: 0.5914 - accuracy: 0.7083
Epoch 1/2
1698/1698 [==============================] - 0s 96us/step - loss: 0.6634 - accuracy: 0.6519
Epoch 2/2
1698/1698 [==============================] - 0s 46us/step - loss: 0.5991 - accuracy: 0.6938
Epoch 1/3
1697/1697 [==============================] - 0s 107us/step - loss: 0.6664 - accuracy: 0.6547
Epoch 2/3
1697/1697 [==============================] - 0s 50us/step - loss: 0.58

In [36]:
best_param = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [37]:
best_param

{'batch_size': 20, 'epochs': 2, 'optimizer': 'rmsprop'}

In [38]:
best_accuracy

0.7010330709011191